TLDR:
-
- we have 534 patients with lateral videos: 25 patients with no lateral videos
- 64 patients with videos from different acquisition dates
- todo: swallows need to be extracted



Analysis of Kopp Dataset (Erlangen / Disphagia)
TLDR: Disphagia dataset with uncut videos of swallows from different perspectives

Pros:
- patient information
- labels for conditions

Challenges / Todos:
- [ ] multiple perspectives (1-20, ~5) -> Identify correct perspectives (heuristic or CNN Classification)
    - perspectives: lateral, frontal, zoomed in lateral and movement doing scan
    - [ ] sometimes >1 video from correct perspective -> choose one
- [ ] uncut videos -> Identify swallows and cut videos
- [ ] Some videos have movements in them (of patient or camera) -> further analysis
- only disphagia, no normals

Questions:
- Swallow extraction
- if we use different acquisition dates, how to handle them? age col needs to be adjusted

### Folder Structure:
- patient id dcm folders
   - 560 patient identifiers
  - every patient folder can have multiple subfolders/videos with dcm file
- Diagnosen und Prozeduren folder
   - Diagnosen_psuedonymisiert.csv
      -  Columns: PATIENT_ID	FALL_ID	ICD_CODE	DIAGNOSE_DATUM	GEWISSHEIT_ID
      - 3,742 rows
  - Prozeduren_pseudonymisiert.csv
      - Columns: PATIENT_ID	FALL_ID	OPS_CODE PROZEDUR_DATUM
      - 1,523 rows
- extracted_videos_and_images folder
   - extracted mp4 files from dcm files per video id
- Kopp_Data_Overview.xlsx
   - columns:
      - Birth Year
      - Age
      - Sex
      - Manufacturer
      - Manufacturer's Model Name
      - Device Serial Number
      - Image/ Video Identifier
      - Comments about the Image/ Video
    -  3,018 rows
    - prob. just extracted info from dicom files
- README.md
   - mentions test sample ids


In [ ]:
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

### get data

In [ ]:
videos_path = r"\\fauad.fau.de\shares\ANKI\Projects\Swallowing\Data\from_Kopp\extracted_videos_and_images"

In [ ]:
# orientation labels
orientation_labels_path = "kopp_video_labels.csv"
df_orientation_labels = pd.read_csv(orientation_labels_path)
df_orientation_labels.head()

In [ ]:
# data overview
overview_path = r"\\fauad.fau.de\shares\ANKI\Projects\Swallowing\Data\from_Kopp\Kopp_Data_Overview.xlsx"
df_overview = pd.read_excel(overview_path)
df_overview.head()

In [ ]:
# diagnosis data
diagnosis_path = r"\\fauad.fau.de\shares\ANKI\Projects\Swallowing\Data\from_Kopp\Diagnosen und Prozeduren\Diagnosen_pseudonymisiert.csv"
df_diagnosis = pd.read_csv(diagnosis_path)
df_diagnosis.head()

### process data

In [ ]:
### merge
df_orientation_labels['video_id'] = df_orientation_labels['video_id'].apply(lambda x: x.split('.')[0])

df_overview = df_overview.merge(df_orientation_labels, left_on='Image/ Video Identifier', right_on='video_id')


In [ ]:
# process date
df_overview['Acquisition Date'] = pd.to_datetime(df_overview['Acquisition Date'], format='%Y%m%d')
df_overview['Acquisition Date'] = df_overview['Acquisition Date'].dt.date
df_overview['Acquisition Date'] = pd.to_datetime(df_overview['Acquisition Date'])

# process age
# for age column, remove last char from string and convert to int
df_overview["Age"] = df_overview["Age"].apply(lambda x: int(x[:-1]))

In [ ]:
# video images folder -> we only are interested in the video files
# get all video files
video_files = [f.split('.')[0] for f in os.listdir(videos_path) if f.endswith('.mp4')]
print(f"Number of video files: {len(video_files)}")

# png files
png_files = [f.split('.')[0] for f in os.listdir(videos_path) if f.endswith('.png')]
print(f"Number of png files: {len(png_files)}")
print(f"Total number of files: {len(video_files) + len(png_files)}")

df_video_files = pd.DataFrame(video_files, columns=['Image/ Video Identifier'])
df_video_files['PatID'] = df_video_files['Image/ Video Identifier'].apply(lambda x: x.split('_')[0])

In [ ]:
# filter: only videos with label 1
df_overview_lateral = df_overview[df_overview["label"] == 1]

### analysis

#### Distribution of orientation labels

In [ ]:
df_orientation_labels["label"].value_counts()

In [ ]:
# plot distribution of orientation labels
df_orientation_labels["label"].value_counts().plot(kind='bar')

In [ ]:
df_overview
# create column that is a link to the video
df_overview['video_link'] = df_overview['Image/ Video Identifier'].apply(lambda x: os.path.join(videos_path, f"{x}.mp4"))
df_overview

plot one sample per class

In [ ]:
# i have 4 classes in the df_overview for column label
# out of each class i want to sample one video and from that get the first frame
# i want to make a plot with classes 1-4 with each having the first frame

# get one video per class
df_overview_sample = df_overview.groupby('label').sample(1)

# List to store video frames and titles
frames = []
titles = []

# Extract the first frame of each video
for video_id in df_overview_sample['video_id']:
    video_path = os.path.join(videos_path, f"{video_id}.mp4")
    if os.path.exists(video_path):
        # Capture the video
        cap = cv2.VideoCapture(video_path)
        success, frame = cap.read()
        if success:
            # Convert BGR to RGB for proper visualization
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
            titles.append(video_id)
        cap.release()

# Create a plot with the frames
fig, axes = plt.subplots(1, len(frames), figsize=(15, 5))
if len(frames) == 1:
    axes = [axes]  # Ensure axes is iterable for single video case
for ax, frame, title in zip(axes, frames, titles):
    ax.imshow(frame)
    ax.set_title(title)
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
from skvideo.io import vread
def save_video_frames_plot(video_path, save_path):
    """
    Display frames 3, 4, and 5 of the video side by side and prompt for a label.
    Save the label to the CSV file.
    """
    # Read the video
    video_id = os.path.basename(video_path)
    video = vread(video_path)

    # Check if video has at least 15 frames
    if len(video) < 15:
        print(f"Video {video_id} does not have enough frames. Skipping.")
        return

    # Extract frames 0, middle and last
    frame_indices = [0, int((len(video)/2)-1), len(video)-1]
    frames = [video[i] for i in frame_indices]

    # Display frames side by side
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    titles = [f"Frame {i+1}" for i in frame_indices]
    for ax, frame, title in zip(axes, frames, titles):
        ax.imshow(frame)
        ax.axis('off')
        ax.set_title(title)

    plt.suptitle(f"Video: {video_id}\n")
    #plt.show()
    full_save_path = os.path.join(save_path, f"{video_id}.png")
    # dont show plot, just save it
    plt.savefig(full_save_path)

df_overview_orientation_unsure = df_overview[df_overview["label"] == 3].head(5)
for video_id in df_overview_orientation_unsure['video_id']:
    video_path = os.path.join(videos_path, f"{video_id}.mp4")
    save_video_frames_plot(video_path, "media/3")

frames distribution for lateral (takes 1min)

In [ ]:
# analyze how many frames we have per video
def get_frame_count(video_path):
    try:
        cap = cv2.VideoCapture(video_path)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Total frame count in the video
        cap.release()
        return frame_count
    except Exception as e:
        print(f"Error processing video {video_path}: {e}")
        return None

# get frame count for all videos and save it as column in df -> takes 1min
df_overview_lateral['Frame Count'] = df_overview_lateral['Image/ Video Identifier'].apply(lambda x: get_frame_count(os.path.join(videos_path, x + '.mp4')))

In [ ]:
# create histogram of frame count
plt.hist(df_overview_lateral['Frame Count'], bins=100)
# add x y axis labels
plt.xlabel("Frame Count")
plt.ylabel("Frequency")
plt.title("Frame Count Distribution per full video")

print("Frame Count Stats:")
print(df_overview_lateral['Frame Count'].describe())

frame_value_counts = df_overview_lateral['Frame Count'].value_counts()

Lateral Video Distribution per patient

In [ ]:
df_videos_per_patient = df_overview_lateral.groupby('PatID').size()
print(df_videos_per_patient.describe())

In [ ]:
# how many video ids do we have per patient - group by PatID
# create histogram
plt.hist(df_overview_lateral.groupby('PatID').size(), bins=50)
# add x y axis labels
plt.xlabel("Number of Videos")
plt.ylabel("Frequency")
plt.title("Number of Lateral Videos per Patient")

In [ ]:
# print out video ids grouped by patient id, where we have more than 1 video
print(df_overview_lateral.groupby('PatID').filter(lambda x: len(x) > 1).groupby('PatID')['Image/ Video Identifier'].apply(list))

In [ ]:
# show video with 19 frames
video_path = os.path.join(videos_path, df_overview_lateral[df_overview_lateral['Frame Count'] == 19].iloc[0]['Image/ Video Identifier'] + '.mp4')
video_path

How many patients have no lateral video?

In [ ]:
# how many patients are there with no label 1 video label
df_overview[df_overview['PatID'].isin(df_overview_lateral['PatID']) == False].groupby('PatID').size()

In [ ]:
df_overview_lateral['PatID'].nunique()

Distribution of Acquisition dates

In [ ]:
# Group by PatID and check for consistency in Acquisition Date
inconsistent_patients_acquisition = df_overview_lateral.groupby('PatID')['Acquisition Date'].nunique()

# Filter for patients with more than one unique acquisition date
inconsistent_acquisitions_patient_ids = inconsistent_patients_acquisition[inconsistent_patients_acquisition > 1].index.tolist()

df_inconsistent_acquisition = df_overview_lateral[df_overview_lateral['PatID'].isin(inconsistent_acquisitions_patient_ids)].groupby('PatID')['Acquisition Date'].agg(['nunique', 'min', 'max'])
df_inconsistent_acquisition['Max Time Difference (days)'] = (df_inconsistent_acquisition['max'] - df_inconsistent_acquisition['min']).dt.days
df_inconsistent_acquisition#.head()

In [ ]:
# how many patients have more than 1 video
print(f"Number of patients with more than 1 video: {len(df_videos_per_patient[df_videos_per_patient > 1])}")
# how many patients have more than 1 video from different acquisition dates
print(f"Number of patients with more than 1 video from different acquisition dates: {len(df_inconsistent_acquisition)}")

In [ ]:
# plot max time difference as histogram
plt.hist(df_inconsistent_acquisition['Max Time Difference (days)'], bins=10)
# add x y axis labels
plt.xlabel("Max Time Difference (days)")
plt.ylabel("Frequency")
plt.title("Max Acquisition Time Difference Distribution per Patient")

In [ ]:
# plot distribution of acquisition dates per year
df_overview_lateral['Acquisition Date'].dt.year.value_counts().sort_index().plot(kind='bar')
# add x y axis labels
plt.xlabel("Year")
plt.ylabel("Frequency")
plt.title("Acquisition Date Distribution per Year")

age

note: age at acquisition is prob. from first acquisition date?

In [ ]:
# plot distribution of age
print("Age Stats:")
print(df_overview_lateral["Age"].describe())

In [ ]:
# Determine the bin range
min_age = df_overview_lateral["Age"].min()
max_age = df_overview_lateral["Age"].max()
bins = range(min_age, max_age + 2, 5)  # Bin width of 2 years

# Plot histogram for all data combined (no gender differentiation)
plt.figure(figsize=(10, 6))
plt.hist(
    df_overview_lateral["Age"],
    bins=bins,
    density=True,
    edgecolor='black'
)
plt.ylim(0, 0.01)  # Adjust based on expected density range
plt.yticks([0, 0.01, 0.02, 0.03, 0.04, 0.05])  # Standardized ticks


# Add titles and labels
plt.title("Normalized Age Distribution")
plt.xlabel("Age")
plt.ylabel("Density")
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.show()

gender distribution

In [ ]:
df_gender = df_overview_lateral[["PatID", "Sex"]].drop_duplicates()
print(df_gender["Sex"].value_counts())
print(round(df_gender["Sex"].value_counts() / len(df_gender), 2))

age distribution per gender


In [ ]:
# Adjust bins to be consistent for both genders
plt.figure(figsize=(10, 6))

# Determine a consistent bin range for both genders
min_age = df_overview_lateral["Age"].min()
max_age = df_overview_lateral["Age"].max()
bins = range(min_age, max_age + 2, 5)  # Bin width of 2 years

for gender in df_overview_lateral["Sex"].unique():
    subset = df_overview_lateral[df_overview_lateral["Sex"] == gender]
    plt.hist(subset["Age"], bins=bins, alpha=0.6, density=True, label=f"{gender}", edgecolor='black')

plt.ylim(0, 0.01)  # Adjust based on expected density range
plt.yticks([0, 0.01, 0.02, 0.03, 0.04, 0.05])  # Standardized ticks

plt.title("Normalized Age Distribution by Gender with Consistent Bins")
plt.xlabel("Age")
plt.ylabel("Density")
plt.legend(title="Gender")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
print(f"Age of Men: {df_overview_lateral[df_overview_lateral["Sex"] == "M"]["Age"].describe()}\n")

In [ ]:
print(f"Age of Women: {df_overview_lateral[df_overview_lateral["Sex"] == "F"]["Age"].describe()}")

### diagnosis + procedures

how many diagnosis per patient

In [ ]:
df_diagnosis.groupby('PATIENT_ID').size().sort_values(ascending=False)

In [ ]:
# histogram of diagnosis per patient
plt.hist(df_diagnosis.groupby('PATIENT_ID').size(), bins=50)

In [ ]:
# show all other columns for group by patient id per patient
df_diagnosis[df_diagnosis["PATIENT_ID"] == 1911001813]#.groupby('PATIENT_ID')

type of diagnosis



In [ ]:
df_diagnosis["FALL_ID"].value_counts()

diagnosis confidence
info:
- ZN    Zustand nach
- VA    Verdachtsdiagnose
- GE    gesicherte Diagnose
- AU    ausgeschlossene Diagnose
- „ein Leerzeichen“
    - -> bei ambulanten Fällen: Annahme gesicherte Diagnose
    - -> bei stationären Fällen: GEWISSHEIT_ID ist immer leer

In [ ]:
df_diagnosis["GEWISSHEIT_ID"].value_counts()

## IQA

video resolution (of first frame)

In [ ]:
# do all the videos have the same resolution?
# go through all the videos first frame and check resolution
resolution = []
for video_id in df_overview_lateral['Image/ Video Identifier']:
    video_path = os.path.join(videos_path, f"{video_id}.mp4")
    if os.path.exists(video_path):
        cap = cv2.VideoCapture(video_path)
        success, frame = cap.read()
        if success:
            resolution.append(frame.shape)
        cap.release()
df_overview_lateral['Resolution'] = resolution

In [ ]:
df_overview_lateral['Resolution'].value_counts()

In [ ]:
# print out video ids with different resolutions
df_overview_lateral[df_overview_lateral['Resolution'] != (1024, 1024, 3)][['Image/ Video Identifier', 'Resolution']]

In [ ]:
## artefacts - black frames present?


dynamic range analysis

In [ ]:
# dynamic range analysis
# = intensity histogram

# plot the intensity histogram of the first frame of a video
video_path = os.path.join(videos_path, df_overview_lateral.iloc[0]['Image/ Video Identifier'] + '.mp4')
cap = cv2.VideoCapture(video_path)
success, frame = cap.read()
if success:
    # Convert to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    plt.hist(gray_frame.ravel(), bins=256, range=[0, 256])
    plt.title("Intensity Histogram of the First Frame")
    plt.xlabel("Pixel Intensity")
    plt.ylabel("Frequency")
    plt.show()
cap.release()




In [ ]:
import os
import cv2
import random
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

# Assuming `df_overview_lateral` and `videos_path` are already defined
video_files = [
    os.path.join(videos_path, f"{video_id}.mp4")
    for video_id in df_overview_lateral['Image/ Video Identifier']
]

# Initialize an empty histogram array with 256 bins
all_histograms = np.zeros(256, dtype=np.float64)  # Use float64 for accumulation

# Process each video
for video_path in tqdm(video_files, desc="Processing Videos"):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # Randomly sample a frame index
    if frame_count > 0:
        random_frame_idx = random.randint(0, frame_count - 1)
        cap.set(cv2.CAP_PROP_POS_FRAMES, random_frame_idx)
        success, frame = cap.read()
        if success:
            # Convert the frame to grayscale
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Compute the histogram for the grayscale frame
            hist, _ = np.histogram(gray_frame.ravel(), bins=256, range=[0, 256])
            all_histograms += hist  # Accumulate histograms

    cap.release()

# Check if any valid histograms were aggregated
if all_histograms.sum() == 0:
    print("No valid frames were processed. Please check the video paths and frame selection.")
else:
    # Normalize the aggregated histogram for visualization
    all_histograms_normalized = all_histograms / all_histograms.sum()

    # Plot the aggregated histogram
    plt.figure(figsize=(10, 6))
    plt.bar(range(256), all_histograms_normalized, width=1, edgecolor='black')
    plt.title("Aggregated Intensity Histogram Across Videos")
    plt.xlabel("Pixel Intensity")
    plt.ylabel("Normalized Frequency")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()


In [ ]:
all_histograms

### niqe score

In [ ]:
from pathlib import Path
import skvideo # note: skvideo is pain to work with -> i use custom version: pip install -e .
import skvideo.measure
from skvideo.io import vread
import numpy as np
np.float = np.float64
np.int = np.int_
import skvideo.measure
from skvideo.io import vread
import numpy as np
from PIL import Image


def compute_video_niqe(video_path, first_frame_only=False):
    """
    Compute the NIQE score for a video, either across all frames or just the first frame.

    Parameters:
    - video_path (str): Path to the video file.
    - first_frame_only (bool): If True, compute NIQE for only the first frame.

    Returns:
    - float: Median NIQE score (or NIQE score for the first frame if `first_frame_only` is True).
    """
    # Load the video
    video = vread(video_path)
    print(f"Processing video: {video_path}")
    #print(f"Original video shape: {video.shape}")

    # Convert to grayscale if not already (use only the first channel)
    if len(video.shape) == 4:  # If the video has channels
        video = video[:, :, :, 0]  # Extract the luminance channel (grayscale)
    #print(f"Grayscale video shape: {video.shape}")

    # Select frames
    if first_frame_only:
        video = video[:1]  # Keep only the first frame
        #print("Computing NIQE for the first frame only.")

    # Compute NIQE scores
    niqe_scores = skvideo.measure.niqe(video)
    #print(f"NIQE Scores: {niqe_scores}")

    # Compute the median score (or single score if first frame only)
    median_score = np.median(niqe_scores)
    #print(f"Median NIQE Score: {median_score}")
    return median_score

def compute_niqe_for_videos_in_path(df, first_frame_only=False):
    """
    Compute NIQE scores for all videos in a directory.

    Parameters:
    - videos_path (str): Path to the directory containing video files.
    - first_frame_only (bool): If True, compute NIQE only for the first frame of each video.

    Returns:
    - dict: A dictionary with video file paths as keys and median NIQE scores as values.
    """

    # Dictionary to store results
    niqe_scores = []
    # Compute NIQE for each video
    for i, row in df.iterrows():
        try:
            abs_path = os.path.join(videos_path, row["video_id"] + '.mp4')
            median_score = compute_video_niqe(abs_path, first_frame_only=first_frame_only)
            niqe_scores.append(median_score)
        except Exception as e:
            print(f"Error processing {video_path}: {e}")
            niqe_scores.append(None)
    df["niqe"] = niqe_scores

    return df

compute_first_frame_only = True  # Set to True to compute NIQE only for the first frame
# Compute NIQE scores for all videos in the directory
results = compute_niqe_for_videos_in_path(df_overview_lateral, first_frame_only=compute_first_frame_only)


In [ ]:
# round niqe scores
results["niqe"] = results["niqe"].round(2)
results["niqe"].describe()

highest niqe

In [ ]:
# plot the first frame of the video with the highest NIQE score
highest_niqe_video = results[results["niqe"] == results["niqe"].max()].iloc[0]
video_path = os.path.join(videos_path, highest_niqe_video["video_id"] + '.mp4')
print(f"Video with highest NIQE score: {video_path}")

# Capture the video
cap = cv2.VideoCapture(video_path)
success, frame = cap.read()
if success:
    # Convert BGR to RGB for proper visualization
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.imshow(frame)
    plt.axis('off')
    plt.title(f"Video: {highest_niqe_video['video_id']}\nNIQE Score: {highest_niqe_video['niqe']}")
    plt.show()
cap.release()


In [ ]:
# plot the first frame of the video with the lowest NIQE score
lowest_niqe_video = results[results["niqe"] == results["niqe"].min()].iloc[0]
video_path = os.path.join(videos_path, lowest_niqe_video["video_id"] + '.mp4')
print(f"Video with lowest NIQE score: {video_path}")

# Capture the video
cap = cv2.VideoCapture(video_path)
success, frame = cap.read()
if success:
    # Convert BGR to RGB for proper visualization
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.imshow(frame)
    plt.axis('off')
    plt.title(f"Video: {lowest_niqe_video['video_id']}\nNIQE Score: {lowest_niqe_video['niqe']}")
    plt.show()